# Part 1: Business Problem (SFO Survey)

- Each year, San Francisco Airport (SFO) conducts a customer satisfaction survey to find out what they are doing well and where they can improve
- The survey gauges satisfaction with SFO facilities, services, and amenities. SFO compares results to previous surveys to discover elements of the guest experience that are not satisfactory
- The 2013 SFO Survey Results consists of customer responses to survey questions and an overall satisfaction rating with the airport
- Whether we could use machine learning to predict a customer's overall response given their responses to the individual questions
- You may think this is not very useful because the customer has already provided an overall rating as well as individual ratings for various aspects of the airport such as parking, food quality and restroom cleanliness. However, we didn't stop at prediction instead we asked the question: 

**What factors drove the customer to give the overall rating?**


**Outline of our data flow:**  
1) Load the dataset
2) Understand the data: Compute statistics and create visualizations to get a better understanding of the data
3) Create Model
4) Evaluate the model
5) Feature Importance: Determine the importance of each of the individual ratings in determining the overall rating by the customer

# Part 2: Load the Data

In [0]:
survey = spark.read.csv("dbfs:/databricks-datasets/sfo_customer_survey/2013_SFO_Customer_Survey.csv", header="true", inferSchema="true")

In [0]:
display(survey)

In [0]:
len(survey.columns)

# Part 3: Understand the Data

In [0]:
survey.printSchema()

- As you can see above there are many questions in the survey including what airline the customer flew on, where do they live, etc. For the purposes of answering the above, we will focus on the Q7A, Q7B, Q7C .. Q7O questions since they directly related to customer satisfaction, which is what we want to measure

- The possible values for the above are:  

0 = no answer, 1 = Unacceptable, 2 = Below Average, 3 = Average, 4 = Good, 5 = Outstanding, 6 = Not visited or not applicable

Select only the fields you are interested in

In [0]:
dataset = survey.select("Q7A_ART", "Q7B_FOOD", "Q7C_SHOPS", "Q7D_SIGNS", "Q7E_WALK", "Q7F_SCREENS", "Q7G_INFOARR", "Q7H_INFODEP", "Q7I_WIFI", "Q7J_ROAD", "Q7K_PARK", "Q7L_AIRTRAIN", "Q7M_LTPARK", "Q7N_RENTAL", "Q7O_WHOLE")

In [0]:
len(dataset.columns)

In [0]:
# Let's start with the overall rating
from pyspark.sql.functions import *
dataset.selectExpr('avg(Q7O_WHOLE) Q7O_WHOLE').take(1)

In [0]:
# The overall rating is only 3.87, so slightly above average. Let's get the averages of the constituent ratings

avgs = dataset.selectExpr('avg(Q7A_ART) Q7A_ART', 'avg(Q7B_FOOD) Q7B_FOOD', 'avg(Q7C_SHOPS) Q7C_SHOPS', 'avg(Q7D_SIGNS) Q7D_SIGNS', 'avg(Q7E_WALK) Q7E_WALK', 'avg(Q7F_SCREENS) Q7F_SCREENS', 'avg(Q7G_INFOARR) Q7G_INFOARR', 'avg(Q7H_INFODEP) Q7H_INFODEP', 'avg(Q7I_WIFI) Q7I_WIFI', 'avg(Q7J_ROAD) Q7J_ROAD', 'avg(Q7K_PARK) Q7K_PARK', 'avg(Q7L_AIRTRAIN) Q7L_AIRTRAIN', 'avg(Q7M_LTPARK) Q7M_LTPARK', 'avg(Q7N_RENTAL) Q7N_RENTAL')
display(avgs)

In [0]:
display(dataset)

# Part 4: Create Model

So basic statistics can't seem to answer the question: **What factors drove the customer to give the overall rating?**

## Replace responce 0 & 6 with average rating

- To treat responses, **0 = No Answer** and **6 = Not Visited or Not Applicable** as missing values
- **First option:** Replace missing values with column mean
- **Second option:** Set all the values of 0 or 6 to the average rating of 3

In [0]:
training = dataset.withColumn("label", dataset['Q7O_WHOLE']*1.0).na.replace(0,3).replace(6,3)

In [0]:
display(training)

## Define the ML pipeline

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
# Defining the input columns
inputCols = ['Q7A_ART', 'Q7B_FOOD', 'Q7C_SHOPS', 'Q7D_SIGNS', 'Q7E_WALK', 'Q7F_SCREENS', 'Q7G_INFOARR', 'Q7H_INFODEP', 'Q7I_WIFI', 'Q7J_ROAD', 'Q7K_PARK', 'Q7L_AIRTRAIN', 'Q7M_LTPARK', 'Q7N_RENTAL']

# Creating a VectorAssembler
va = VectorAssembler(inputCols=inputCols,outputCol="features")

# Creating a DecisionTreeRegressor
dt = DecisionTreeRegressor(labelCol="label", featuresCol="features", maxDepth=4)

# Creating a RegressionEvaluator
evaluator = RegressionEvaluator(metricName = "rmse", labelCol="label")

# Creating a ParamGridBuilder 
grid = ParamGridBuilder().addGrid(dt.maxDepth, [3, 5, 7, 10]).build()

# Creating a CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=grid, evaluator=evaluator, numFolds = 10)

# Creating a Pipeline
pipeline = Pipeline(stages=[va, dt])

## Train the Model

In [0]:
model = pipeline.fit(training)

## View the tree

In [0]:
display(model.stages[-1])

## Make predictions

In [0]:
predictions = model.transform(training)
display(predictions)

# Part 5: Evaluate the model

## Validate the model using root mean squared error (RMSE)

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator()

evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})

## Save the model

In [0]:
import uuid
model_save_path = f"/tmp/sfo_survey_model/{str(uuid.uuid4())}"
model.write().overwrite().save(model_save_path)

# Part 6: Feature Importance

In [0]:
# feature importances
model.stages[1].featureImportances

In [0]:
# map the features to their proper names to make them easier to read

featureImportance = model.stages[1].featureImportances.toArray()
featureNames = map(lambda s: s.name, dataset.schema.fields)
featureImportanceMap = zip(featureImportance, featureNames)

In [0]:
print(featureImportanceMap)

In [0]:
# Convert featureImportanceMap to a Dataframe

importancesDf = spark.createDataFrame(sc.parallelize(featureImportanceMap).map(lambda r: [r[1], float(r[0])]))

In [0]:
display(importancesDf)

In [0]:
# Rename column names
importancesDf = importancesDf.withColumnRenamed("_1", "Feature").withColumnRenamed("_2", "Importance")

In [0]:
display(importancesDf)

In [0]:
# visulization of the Feature Importances (pie chart)
display(importancesDf.orderBy(desc("Importance")))

In [0]:
# visulization of the Feature Importances (bar chart)
display(importancesDf.orderBy(desc("Importance")))

In [0]:
importancesDf.orderBy(desc("Importance")).limit(3).agg(sum("Importance")).take(1)


The 3 most important features are:

1. Signs 
2. Screens
3. Food

These 3 features combine to make up 88% of the overall rating

In [0]:
# delete saved model
dbutils.fs.rm(model_save_path, True)